In [1]:
# %%
# 0. INSTALL DEPENDENCIES
%pip install exifread Pillow

# %% [markdown]
# # 🕵️ Digital Forensics: Metadata Analysis
# This notebook inspects image metadata to find traces of editing software.

# %%
import exifread
from PIL import Image, PngImagePlugin
import os

# %%
# 1. CREATE TEST IMAGES
# We create one "Clean" image and one "Fake" image to test our logic.

# Create a clean dummy image
img = Image.new('RGB', (100, 100), color = 'green')
img.save("test_clean.jpg")

# Create a fake image with injected metadata
meta = PngImagePlugin.PngInfo()
meta.add_text("Software", "Adobe Photoshop 2024") # 🚨 The smoking gun!
img_fake = Image.new('RGB', (100, 100), color = 'red')
img_fake.save("test_fake.png", pnginfo=meta)

print("✅ Created test files: test_clean.jpg, test_fake.png")

# %%
# 2. ANALYSIS FUNCTION
def analyze_metadata(image_path):
    print(f"\n🔍 Analyzing: {image_path}")
    suspicious = False
    
    try:
        # METHOD A: Check EXIF (Standard for JPEGs)
        with open(image_path, 'rb') as f:
            tags = exifread.process_file(f)
            if 'Image Software' in tags:
                print(f"   [EXIF] Software: {tags['Image Software']}")
                if "Photoshop" in str(tags['Image Software']):
                    suspicious = True

        # METHOD B: Check Pillow Info (Works for PNGs)
        img_obj = Image.open(image_path)
        if "Software" in img_obj.info:
            software = img_obj.info["Software"]
            print(f"   [PNG Info] Software: {software}")
            if "Photoshop" in software or "GIMP" in software:
                suspicious = True
                
    except Exception as e:
        print(f"   Error reading metadata: {e}")

    if suspicious:
        print("🚨 FRAUD ALERT: Image editing software detected!")
    else:
        print("✅ Clean: No suspicious tags found.")

# %%
# 3. RUN THE TEST
analyze_metadata("test_clean.jpg")
analyze_metadata("test_fake.png")

# %%
# 4. CLEANUP
# Delete the test files so we don't clutter the folder
if os.path.exists("test_clean.jpg"): os.remove("test_clean.jpg")
if os.path.exists("test_fake.png"): os.remove("test_fake.png")
print("\n🧹 Cleanup complete.")

     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ---------------------------------------- 59.7/59.7 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
✅ Created test files: test_clean.jpg, test_fake.png

🔍 Analyzing: test_clean.jpg
✅ Clean: No suspicious tags found.

🔍 Analyzing: test_fake.png



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
PNG file does not have exif data.


   [PNG Info] Software: Adobe Photoshop 2024
🚨 FRAUD ALERT: Image editing software detected!

🧹 Cleanup complete.
